In [2]:
import pandas as pd
import numpy as np

In [2]:
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'

sdc = pd.read_csv(ENTREPOT_PATH + 'sdc.csv')
synth = pd.read_csv(ENTREPOT_PATH + 'synthetise.csv')
sdc_r = pd.read_csv(ENTREPOT_PATH + 'sdc_realise_performance.csv')
synth_s = pd.read_csv(ENTREPOT_PATH + 'synthetise_synthetise_performance.csv')

In [ ]:
# Fusion infos synthétisés et synthetise performance
synth = synth[['id','campagnes','sdc_id']].rename(columns={'id' : 'synthetise_id'})
synth_s = synth_s.merge(synth, on='synthetise_id') #244 synthétisés pour lesquels on ne retrouve pas de performances

del(synth)

# Fusion des performances échelle sdc/synthétisé entre réalisé et synthétisé
df = pd.concat([sdc_r,synth_s])
df['entite_id'] = np.where(df['approche_de_calcul'] == 'réalisé', df['sdc_id'], df['synthetise_id'])

del(sdc_r, synth_s)

In [ ]:
# Merge des données de contexte (sdc) et de performances (R + S)
sdc = sdc.rename(columns={'id':'sdc_id'})

df = df.merge(sdc, on='sdc_id', how = 'left').set_index('entite_id')

df = df.rename(columns={'ift_histo_chimique_tot' : 'ift',
                        'approche_de_calcul' : 'mode_de_calcul',
                        'code_dephy' : 'num_dephy'})
df['annees'] = np.where(df['mode_de_calcul'] == 'réalisé', df['campagne'], df['campagnes'])

del(sdc)

In [ ]:
# Filtre des numéro DEPHY bizarres


In [ ]:
# Petit soucis de duplicats (2 realise_id qui sont dupliqués deux et trois fois)
# df = df[df.sdc_id.isin([
#     'fr.inra.agrosyst.api.entities.GrowingSystem_ba5772e6-5521-4e0e-b55f-dd057ed66e21', # qu'avec des NA
#     'fr.inra.agrosyst.api.entities.GrowingSystem_5b32e95f-629a-4ff8-9e6e-7ab23559f233' # plutot correct on regarde apres
#     ])]
df.loc[df['sdc_id'].duplicated()]